# Parquet file parser

In [178]:
import pyarrow as pa
import pyarrow.parquet as pq
import geopandas as gpd
from shapely import wkb
import struct
import io
import os

In [7]:
import parquet_tools

In [9]:
from parquet_tools.gen_py.parquet.ttypes import FileMetaData
from thrift.protocol import TCompactProtocol
from thrift.transport import TTransport

In [225]:
us_places = gpd.read_parquet("../../data/overture/us_places_bkgh.parquet")

In [227]:
us_places = gpd.read_parquet("../../data/overture/us_places_gh_sorted.parquet")

In [228]:
us_places

,id,names_value,main_cat,geometry
91871,102161692214702,Punta Abreojos Beach,beach,POINT (-113.57394 26.71194)
188347,196023190443592,Punta de Abreojos,landmark_and_historical_building,POINT (-113.57222 26.71444)
91870,495699517164768,Antojitos Juanita,mexican_restaurant,POINT (-113.57380 26.71212)
210704,304691419668555,escuela secundaria Francisco Estrada Lucero,public_school,POINT (-113.57345 26.71839)
91872,101056582183398,Punta Abreojos Baja California Sur,None,POINT (-113.57535 26.70866)
...,...,...,...,...
118819,954853131255660,"Baie de Foin (vik i Kanada, lat 49,99, long -6...",landmark_and_historical_building,POINT (-66.93296 49.99449)
118818,161813097519492,Baie à Corbey,landmark_and_historical_building,POINT (-66.93888 49.99128)
118817,1494320604208349,Îles de Mai,landmark_and_historical_building,POINT (-66.96268 49.92729)
854903,897216883680862,Lac des Grâces,lake,POINT (-67.09437 49.99532)


In [171]:
us_places.to_parquet("../../data/overture/us_places_rg_4k.parquet",row_group_size=4096)

,id,names_value,main_cat,geometry
1308150,49679874639,Back Alley Bakery,bakery,POINT (-98.38904 40.58494)
1308151,104889374747817,Smitty's MHP,campground,POINT (-98.38082 40.57477)
1308152,115690661823498,Hoskins Auto Sales,car_dealer,POINT (-98.39011 40.56764)
1308153,562949964809758,Edward Jones - Financial Advisor: Phil Nelson,None,POINT (-98.38725 40.58950)
1308154,133754580009931,South Central Behavioral Services,counseling_and_mental_health,POINT (-98.39292 40.57619)
...,...,...,...,...
1312241,821375404689359,Pilger Senior Center,community_center,POINT (-97.05418 42.00879)
1312242,982922781732498,Foldoy Water Conditioning,water_treatment_equipment_and_services,POINT (-97.05087 42.00593)
1312243,1468645823445485,ASPM Landscapes,landscaping,POINT (-97.36792 42.01883)
1312244,562949961972115,U-Haul Neighborhood Dealer,truck_rentals,POINT (-97.37911 42.01823)


In [2]:
# import zlib

# import snappy

In [5]:
#pq.read_metadata('../../data/overture/us_buildings.parquet')

In [172]:
#file_path = 'sample.parquet'
file_path = '../../data/overture/us_places_rg_4k.parquet'

In [179]:
file_size = os.path.getsize(file_path)

In [180]:
file_size

819276411

In [182]:
with open(file_path, 'rb') as f:
    f.seek(file_size-8)
    footer_length = struct.unpack('<I', f.read(4))[0]
    f.seek(file_size-footer_length - 8)
    footer = f.read(footer_length)
    

In [184]:
len(footer)

2067960

In [185]:
with open(file_path, 'rb') as f:
    f.seek(-8, 2)
    footer_length = struct.unpack('<I', f.read(4))[0]
    f.seek(-footer_length - 8, 2)
    footer_bytes = f.read(footer_length)

In [259]:
len(footer_bytes)

2067960

In [266]:
transportIn = TTransport.TMemoryBuffer(footer_bytes)
protocolIn = TCompactProtocol.TCompactProtocol(transportIn)

In [267]:
fmd = FileMetaData()
fmd.read(protocolIn)

In [268]:
unique_group_ids = [0,5,25]

In [269]:
modified_row_groups = [fmd.row_groups[rg_idx] for rg_idx in unique_group_ids]

In [272]:
modified_row_groups[0].file_offset

4

In [245]:
def local_cat_offset_length(fpath,offset,length):
    with open(fpath, 'rb') as f:
        # Seek to the start of the row group
        f.seek(offset)
        # Read the row group data
        return f.read(length)

In [270]:
last_pos = 4
rg_buckets=[]
for rg in modified_row_groups:
    rg_start_pos = rg.columns[0].meta_data.dictionary_page_offset
    rg_end_pos = rg.columns[len(rg.columns)-1].file_offset
    print(rg_end_pos-rg_start_pos)
    row_group_raw = local_cat_offset_length(file_path, rg_start_pos, rg_end_pos - rg_start_pos)
    rg_buckets.append(row_group_raw)
    overhead_offset = rg_start_pos - last_pos
    rg.file_offset -= overhead_offset
    for i in range(len(rg.columns)):
        rg.columns[i].file_offset -= overhead_offset
        rg.columns[i].meta_data.data_page_offset -= overhead_offset
        rg.columns[i].meta_data.dictionary_page_offset -= overhead_offset
    last_pos += rg_end_pos - rg_start_pos
fmd.row_groups = modified_row_groups
row_group_all = b''.join(rg_buckets)

240737
241332
240116


In [273]:
fmd.row_groups

[RowGroup(columns=[ColumnChunk(file_path=None, file_offset=61657, meta_data=ColumnMetaData(type=6, encodings=[0, 3, 8], path_in_schema=['id'], codec=1, num_values=4096, total_uncompressed_size=84504, total_compressed_size=61653, key_value_metadata=None, data_page_offset=55433, index_page_offset=None, dictionary_page_offset=4, statistics=Statistics(max=None, min=None, null_count=0, distinct_count=None, max_value=b'999420136908323', min_value=b'100107862262047'), encoding_stats=[PageEncodingStats(page_type=2, encoding=0, count=1), PageEncodingStats(page_type=0, encoding=8, count=1)], bloom_filter_offset=None), offset_index_offset=None, offset_index_length=None, column_index_offset=None, column_index_length=None, crypto_metadata=None, encrypted_column_metadata=None), ColumnChunk(file_path=None, file_offset=134165, meta_data=ColumnMetaData(type=6, encodings=[0, 3, 8], path_in_schema=['names_value'], codec=1, num_values=4096, total_uncompressed_size=106868, total_compressed_size=72416, key_

In [274]:
#wrap new footer to out_bytes
transport = TTransport.TMemoryBuffer()
protocolOut = TCompactProtocol.TCompactProtocol(transport)
fmd.write(protocolOut)
out_bytes = transport.getvalue()

PARQUET_MAGIC_BYTES = b'PAR1'
footer_length = len(out_bytes)

# Create the complete file bytes
complete_parquet_bytes = (
    PARQUET_MAGIC_BYTES+
    row_group_all +                # Row group bytes
    out_bytes +                   # Footer bytes
    footer_length.to_bytes(4, 'little') +  # Footer length in little endian
    PARQUET_MAGIC_BYTES              # Parquet magic bytes
)


gpd.read_parquet(io.BytesIO(complete_parquet_bytes))

,id,names_value,main_cat,geometry
1308150,49679874639,Back Alley Bakery,bakery,POINT (-98.38904 40.58494)
1308151,104889374747817,Smitty's MHP,campground,POINT (-98.38082 40.57477)
1308152,115690661823498,Hoskins Auto Sales,car_dealer,POINT (-98.39011 40.56764)
1308153,562949964809758,Edward Jones - Financial Advisor: Phil Nelson,None,POINT (-98.38725 40.58950)
1308154,133754580009931,South Central Behavioral Services,counseling_and_mental_health,POINT (-98.39292 40.57619)
...,...,...,...,...
1424232,705160352954623,Iglesia Revelación 19,religious_organization,POINT (-122.52878 45.63709)
1424233,154875714551392,Les Schwab Tire Center,tire_dealer_and_repair,POINT (-122.55143 45.61963)
1424234,116012121752603,Dennys Restaurant,american_restaurant,POINT (-122.56601 45.66357)
1424235,1970324838942869,Barron Landscaping Inc,landscape_architect,POINT (-122.59094 45.66951)


In [160]:
rg_idx = 25

In [161]:
rg = fmd.row_groups[rg_idx]
rg_start_pos = rg.columns[0].meta_data.dictionary_page_offset
rg_end_pos = rg.columns[len(rg.columns)-1].file_offset
with open(file_path, 'rb') as f:
    # Seek to the start of the row group
    f.seek(rg_start_pos)
    # Read the row group data
    row_group_raw = f.read(rg_end_pos-rg_start_pos)

In [162]:
overhead_offset = rg_start_pos - 4
rg.file_offset -= overhead_offset
for i in range(len(rg.columns)):
    rg.columns[i].file_offset -= overhead_offset
    rg.columns[i].meta_data.data_page_offset -= overhead_offset
    rg.columns[i].meta_data.dictionary_page_offset -= overhead_offset

In [163]:
fmd.row_groups = [rg]

In [ ]:
fmd.row_groups = 

In [ ]:
#wrap new footer to out_bytes
transport = TTransport.TMemoryBuffer()
protocolOut = TCompactProtocol.TCompactProtocol(transport)
fmd.write(protocolOut)
out_bytes = transport.getvalue()

PARQUET_MAGIC_BYTES = b'PAR1'
footer_length = len(out_bytes)

# Create the complete file bytes
complete_parquet_bytes = (
    PARQUET_MAGIC_BYTES+
    row_group_raw +                # Row group bytes
    out_bytes +                   # Footer bytes
    footer_length.to_bytes(4, 'little') +  # Footer length in little endian
    PARQUET_MAGIC_BYTES              # Parquet magic bytes
)

#raw_buffer = pa.BufferReader(row_group_data)
#new pseudo parquet file
file_buffer = pa.BufferReader(complete_parquet_bytes)


In [233]:
len(complete_parquet_bytes)

125163145

In [ ]:
type(complete_parquet_bytes)

In [ ]:
parq_io = io.BytesIO(complete_parquet_bytes)
gpd.read_parquet(parq_io)

In [166]:
df = pq.ParquetFile(file_buffer).read().to_pandas()


In [167]:
gdf = gpd.GeoDataFrame(df,geometry=df['geometry'].apply(wkb.loads),crs="EPSG:4326")

In [168]:
gdf

,id,geometry
9851030,08b44ad0a641efff020026b146eab62e,"POLYGON ((-81.48597 27.40076, -81.48609 27.400..."
9851031,08b44ad0a64f4fff0200e6abd8106c1b,"POLYGON ((-81.48599 27.40163, -81.48599 27.401..."
9851032,08b44ad0a64f1fff0200a2dd663b6631,"POLYGON ((-81.48603 27.40235, -81.48601 27.402..."
9851033,08b44ad0a6283fff0200a703445fd2bf,"POLYGON ((-81.49608 27.40910, -81.49615 27.409..."
9851034,08b44ad0a6615fff0200ce44c5611973,"POLYGON ((-81.49014 27.40736, -81.49035 27.407..."
...,...,...
10899601,08b4450b2d656fff02002bc8a8388e5b,"POLYGON ((-88.72764 30.54624, -88.72758 30.546..."
10899602,08b4450b2d656fff0200e7c74a433ab4,"POLYGON ((-88.72760 30.54643, -88.72760 30.546..."
10899603,08b4450b2d6f5fff0200df2a73df1ced,"POLYGON ((-88.72580 30.54636, -88.72581 30.546..."
10899604,08b4450b2d6f5fff0200538004ff9680,"POLYGON ((-88.72564 30.54639, -88.72554 30.546..."


In [154]:
#preview
#convert wkb to geometry
#df['geometry'].apply(wkb.loads)

In [207]:
file_path = '../../data/overture/us_places_rg_4k.parquet'
parquet_file = pq.ParquetFile(file_path)

In [224]:
parquet_file = pq.ParquetFile(file_path)
row_group_size = parquet_file.metadata.row_group(0).num_rows
rows = parquet_file.metadata.num_rows
[ (i // row_group_size, i % row_group_size) for i in range(rows)]

[(0, 0),
 (0, 1),
 (0, 2),
 (0, 3),
 (0, 4),
 (0, 5),
 (0, 6),
 (0, 7),
 (0, 8),
 (0, 9),
 (0, 10),
 (0, 11),
 (0, 12),
 (0, 13),
 (0, 14),
 (0, 15),
 (0, 16),
 (0, 17),
 (0, 18),
 (0, 19),
 (0, 20),
 (0, 21),
 (0, 22),
 (0, 23),
 (0, 24),
 (0, 25),
 (0, 26),
 (0, 27),
 (0, 28),
 (0, 29),
 (0, 30),
 (0, 31),
 (0, 32),
 (0, 33),
 (0, 34),
 (0, 35),
 (0, 36),
 (0, 37),
 (0, 38),
 (0, 39),
 (0, 40),
 (0, 41),
 (0, 42),
 (0, 43),
 (0, 44),
 (0, 45),
 (0, 46),
 (0, 47),
 (0, 48),
 (0, 49),
 (0, 50),
 (0, 51),
 (0, 52),
 (0, 53),
 (0, 54),
 (0, 55),
 (0, 56),
 (0, 57),
 (0, 58),
 (0, 59),
 (0, 60),
 (0, 61),
 (0, 62),
 (0, 63),
 (0, 64),
 (0, 65),
 (0, 66),
 (0, 67),
 (0, 68),
 (0, 69),
 (0, 70),
 (0, 71),
 (0, 72),
 (0, 73),
 (0, 74),
 (0, 75),
 (0, 76),
 (0, 77),
 (0, 78),
 (0, 79),
 (0, 80),
 (0, 81),
 (0, 82),
 (0, 83),
 (0, 84),
 (0, 85),
 (0, 86),
 (0, 87),
 (0, 88),
 (0, 89),
 (0, 90),
 (0, 91),
 (0, 92),
 (0, 93),
 (0, 94),
 (0, 95),
 (0, 96),
 (0, 97),
 (0, 98),
 (0, 99),
 (0, 100),

In [221]:
parquet_file.metadata.num_rows

13867406

In [202]:
parquet_file.read_row_group(0).to_pandas()

,id,names_value,main_cat,geometry
1308150,49679874639,Back Alley Bakery,bakery,b'\x01\x01\x00\x00\x00\xd23b\x0b\xe6\x98X\xc0R...
1308151,104889374747817,Smitty's MHP,campground,b'\x01\x01\x00\x00\x00jj\xd9Z_\x98X\xc0tf\x16\...
1308152,115690661823498,Hoskins Auto Sales,car_dealer,b'\x01\x01\x00\x00\x00\xa5/\x84\x9c\xf7\x98X\x...
1308153,562949964809758,Edward Jones - Financial Advisor: Phil Nelson,None,b'\x01\x01\x00\x00\x00\xb1!\xee\xa0\xc8\x98X\x...
1308154,133754580009931,South Central Behavioral Services,counseling_and_mental_health,b'\x01\x01\x00\x00\x00p|\xed\x99%\x99X\xc0 $\x...
...,...,...,...,...
1312241,821375404689359,Pilger Senior Center,community_center,b'\x01\x01\x00\x00\x00\x10\x9e\xbf\xbfwCX\xc0~...
1312242,982922781732498,Foldoy Water Conditioning,water_treatment_equipment_and_services,b'\x01\x01\x00\x00\x00\x0e\x12\xa2|ACX\xc0+\x8...
1312243,1468645823445485,ASPM Landscapes,landscaping,b'\x01\x01\x00\x00\x00\xe2\x1aD\x10\x8cWX\xc0\...
1312244,562949961972115,U-Haul Neighborhood Dealer,truck_rentals,b'\x01\x01\x00\x00\x001\xcbCKCXX\xc0\xd4\xb5\x...


In [203]:
us_places.head(4096)

,id,names_value,main_cat,geometry
1308150,49679874639,Back Alley Bakery,bakery,POINT (-98.38904 40.58494)
1308151,104889374747817,Smitty's MHP,campground,POINT (-98.38082 40.57477)
1308152,115690661823498,Hoskins Auto Sales,car_dealer,POINT (-98.39011 40.56764)
1308153,562949964809758,Edward Jones - Financial Advisor: Phil Nelson,None,POINT (-98.38725 40.58950)
1308154,133754580009931,South Central Behavioral Services,counseling_and_mental_health,POINT (-98.39292 40.57619)
...,...,...,...,...
1312241,821375404689359,Pilger Senior Center,community_center,POINT (-97.05418 42.00879)
1312242,982922781732498,Foldoy Water Conditioning,water_treatment_equipment_and_services,POINT (-97.05087 42.00593)
1312243,1468645823445485,ASPM Landscapes,landscaping,POINT (-97.36792 42.01883)
1312244,562949961972115,U-Haul Neighborhood Dealer,truck_rentals,POINT (-97.37911 42.01823)


In [201]:
(parquet_file.read_row_group(0).to_pandas() == us_places.head(4096))

,id,names_value,main_cat,geometry
1308150,True,True,True,False
1308151,True,True,True,False
1308152,True,True,True,False
1308153,True,True,False,False
1308154,True,True,True,False
...,...,...,...,...
1312241,True,True,True,False
1312242,True,True,True,False
1312243,True,True,True,False
1312244,True,True,True,False


In [ ]:
file_metadata = parquet_file.metadata
row_group_metadata = file_metadata.row_group(0)

In [4]:
type(file_metadata.metadata)

dict

In [ ]:
file_metadata.row_group

In [ ]:
file_metadata.metadata

In [27]:
file_metadata.write_metadata_file('meta')

In [ ]:
parquet_file.read_row_group(0)

In [69]:
row_group_start = row_group_metadata.column(0).dictionary_page_offset
row_group_end = row_group_metadata.column(row_group_metadata.num_columns-1).file_offset

In [70]:
row_group_end

7977

In [5]:
with open(file_path, 'rb') as f:
    # Seek to the start of the row group
    f.seek(row_group_start)

    # Read the row group data
    row_group_data = f.read(row_group_end-row_group_start)

    f.seek(-8, 2)
    footer_length = struct.unpack('<I', f.read(4))[0]
    f.seek(-footer_length - 8, 2)
    footer = f.read(footer_length)

In [6]:
footer_length

14702

In [21]:
PARQUET_MAGIC_BYTES = b'PAR1'
footer_length = len(footer)

# Create the complete file bytes
complete_parquet_bytes = (
    PARQUET_MAGIC_BYTES+
    row_group_data +                # Row group bytes
    footer +                   # Footer bytes
    footer_length.to_bytes(4, 'little') +  # Footer length in little endian
    PARQUET_MAGIC_BYTES              # Parquet magic bytes
)

#raw_buffer = pa.BufferReader(row_group_data)

file_buffer = pa.BufferReader(complete_parquet_bytes)

# Read the footer to get the schema and other metadata
metadata = pq.read_metadata(file_buffer)
schema = metadata.schema.to_arrow_schema()


In [22]:
raw_buffer

<pyarrow.BufferReader closed=False own_file=False is_seekable=True is_writable=False is_readable=True>

In [14]:
metadata.schema

required group field_id=-1 schema {
  optional binary field_id=-1 id (String);
  optional binary field_id=-1 geometry;
  optional int64 field_id=-1 __index_level_0__;
}

In [11]:
# import pyarrow as pa
# import pyarrow.parquet as pq
# import pandas as pd

# def read_row_group_bytes(row_group_bytes, footer_bytes):
#     # Length of the footer
#     footer_length = len(footer_bytes)

#     # Create the complete file bytes
#     complete_footer_bytes = (
#         row_group_bytes +                # Row group bytes
#         footer_bytes +                   # Footer bytes
#         footer_length.to_bytes(4, 'little') +  # Footer length in little endian
#         PARQUET_MAGIC_BYTES              # Parquet magic bytes
#     )

#     footer_buffer = pa.BufferReader(complete_footer_bytes)

#     # Read the footer to get the schema and other metadata
#     metadata = pq.read_metadata(footer_buffer)
#     schema = metadata.schema.to_arrow_schema()

#     # Create a ParquetFile object using the footer and row group data
#     parquet_file = pq.ParquetFile(pa.BufferInputStream(row_group_bytes), metadata=metadata)

#     # Read the row group data into a table
#     table = parquet_file.read()

#     # Convert the PyArrow Table to a Pandas DataFrame
#     df = table.to_pandas()

#     return df

In [ ]:
pq.read_table()

In [23]:
pq.read_table(raw_buffer)

ArrowInvalid: Could not open Parquet input source '<Buffer>': Parquet magic bytes not found in footer. Either the file is corrupted or this is not a parquet file.

In [124]:
df = pq.ParquetFile(file_buffer).read().to_pandas()

OSError: Invalid column metadata (corrupt file?)

In [89]:
df.iloc[0].geometry

b'\x01\x01\x00\x00\x00\xed\x83,\x0b&\x91^\xc0\xa9y\xd8U\x92\xc7B@'

In [ ]:
metadata.to_dict()

In [95]:
type(df)

pandas.core.frame.DataFrame

In [92]:
df['geometry'].apply(wkb.loads)

7295075      POINT (-122.267947 37.5591533)
9884609      POINT (-95.2529326 32.3175498)
9346720      POINT (-96.9693426 32.6577803)
7723038     POINT (-117.6901221 33.9781458)
9211371      POINT (-97.1767072 31.5412061)
10166286    POINT (-104.7552143 38.8632789)
10342841    POINT (-104.9605377 39.6769681)
6627234     POINT (-117.1037212 33.0179705)
8947394      POINT (-98.4872116 29.4669053)
9537053         POINT (-94.6335299 29.4593)
Name: geometry, dtype: object

In [56]:
read_row_group_bytes(row_group_data,footer)

AttributeError: module 'pyarrow' has no attribute 'BufferInputStream'

In [53]:
footer_buffer = pa.BufferReader(footer)

In [54]:
pq.read_metadata(footer_buffer)

ArrowInvalid: Parquet magic bytes not found in footer. Either the file is corrupted or this is not a parquet file.

In [51]:
df = read_row_group_bytes(row_group_data, footer)

ArrowInvalid: Parquet magic bytes not found in footer. Either the file is corrupted or this is not a parquet file.

In [44]:
class ParquetBuffer(io.BytesIO):
    def __init__(self, row_group_data, file_metadata):
        self.row_group_data = row_group_data
        self.file_metadata = file_metadata
        
        # Construct the Parquet file structure
        self.construct_file()
        
    def construct_file(self):
        # Start with Parquet magic number
        self.write(b'PAR1')
        
        # Write column data
        data_start = self.tell()
        self.write(self.row_group_data)
        data_end = self.tell()
        
        # Write file metadata
        file_metadata_start = self.tell()
        self.write(self.file_metadata)
        file_metadata_end = self.tell()
        
        # Write metadata size and Parquet magic number
        metadata_size = file_metadata_end - file_metadata_start
        self.write(struct.pack('<I', metadata_size))
        self.write(b'PAR1')
        
        # Seek to the beginning of the buffer
        self.seek(0)

In [45]:
p_buf = ParquetBuffer(row_group_data, footer)

In [48]:
pq.ParquetFile(p_buf).read_row_group(0)

OSError: Corrupt snappy compressed data.

In [11]:
# Create a PyArrow buffer from the read data
buffer = pa.py_buffer(row_group_data)

In [12]:
schema.to_arrow_schema()

id: string
geometry: binary
bbox: struct<xmax: double, xmin: double, ymax: double, ymin: double>
  child 0, xmax: double
  child 1, xmin: double
  child 2, ymax: double
  child 3, ymin: double
version: int32
update_time: string
sources: list<element: struct<confidence: null, dataset: string, property: string, record_id: string>>
  child 0, element: struct<confidence: null, dataset: string, property: string, record_id: string>
      child 0, confidence: null
      child 1, dataset: string
      child 2, property: string
      child 3, record_id: string
names: struct<common: null, primary: string, rules: null>
  child 0, common: null
  child 1, primary: string
  child 2, rules: null
categories: struct<alternate: list<element: string>, main: string>
  child 0, alternate: list<element: string>
      child 0, element: string
  child 1, main: string
confidence: double
websites: list<element: string>
  child 0, element: string
socials: list<element: string>
  child 0, element: string
emails: 

In [64]:
metadata

  created_by: parquet-cpp-arrow version 15.0.2
  num_columns: 29
  num_rows: 10
  num_row_groups: 1
  format_version: 2.6
  serialized_size: 14702

In [13]:
meta_dict = metadata.to_dict()

In [ ]:
meta_dict

In [ ]:
pq.FileMetaData(meta_dict)

In [1]:
metadata

NameError: name 'metadata' is not defined

In [66]:
# Read the table from the buffer
pf = pq.ParquetFile(buffer,metadata=metadata)

# Convert the PyArrow Table to a Pandas DataFrame
df = pf.read()

OSError: Invalid column metadata (corrupt file?)

In [ ]:
row_group_metadata = footer.row_group(row_group_index)

In [6]:
# Get the start and end positions of the row group
start_pos = row_group.column(0).file_offset
end_pos = start_pos

In [7]:
start_pos

444

In [8]:
columns = []
file = open(file_path, 'rb')
column_metadata = row_group.column(0)

In [26]:
type(column_metadata)

pyarrow._parquet.ColumnChunkMetaData

In [17]:
# Seek to the start of the column chunk
file.seek(column_metadata.file_offset)

# Read the column chunk
column_data = file.read(column_metadata.total_compressed_size)

# Create a buffer from the column data
buf = pa.py_buffer(column_data)

# Create an Arrow input stream
stream = pa.BufferReader(buf)

In [40]:
column_metadata = pq.ColumnChunkMetaData(column_metadata,
        data_page_offset=0,
        dictionary_page_offset=None,
        index_page_offset=None,
        has_dictionary_page=False,
)

# Create file metadata
schema = pa.schema([pa.field(column_schema.name, column_schema.physical_type)])
file_metadata = pq.FileMetaData(schema, len(column_data), 1)

NameError: name 'column_schema' is not defined

In [ ]:
row_group.to_dict()['columns']

In [ ]:
def read_int32(file):
    return struct.unpack('<i', file.read(4))[0]

def read_int64(file):
    return struct.unpack('<q', file.read(8))[0]

def read_page_header(file):
    header_bytes = file.read(4)
    header_size = struct.unpack('<i', header_bytes)[0]
    header_data = file.read(header_size)
    return read_thrift(io.BytesIO(header_data))

def read_column_values(file_path, row_group_index=0, column_index=0):
    # Read file metadata
    parquet_file = pq.ParquetFile(file_path)
    metadata = parquet_file.metadata
    with open(file_path, 'rb') as file:
        # Get column chunk metadata
        row_group = metadata.row_groups[row_group_index]
        column_chunk = row_group.columns[column_index]

        # Seek to the start of the column chunk
        file.seek(column_chunk.file_offset)

        values = []
        bytes_read = 0
        
        while bytes_read < column_chunk.meta_data.total_compressed_size:
            # Read page header
            page_header = read_page_header(file)
            bytes_read += page_header.compressed_page_size + 4 + len(page_header.thrift_buffer)

            if page_header.type == 0:  # DATA_PAGE
                # Read page data
                page_data = file.read(page_header.compressed_page_size)
                
                # Decompress if necessary
                if column_chunk.meta_data.codec != 0:  # 0 means UNCOMPRESSED
                    # You'll need to implement decompression based on the codec used
                    # For simplicity, we'll assume uncompressed data in this example
                    pass

                # Parse values (this is a simplified version, assuming int32 values)
                # You'll need to adjust this based on your actual data type
                num_values = page_header.data_page_header.num_values
                for _ in range(num_values):
                    value = read_int32(io.BytesIO(page_data))
                    values.append(value)

    return values

# Usage
file_path = 'path/to/your/parquet/file.parquet'
column_values = read_column_values(file_path)
print(column_values)

In [30]:
column_metadata

  file_offset: 444
  file_path: 
  physical_type: BYTE_ARRAY
  num_values: 10
  path_in_schema: id
  is_stats_set: True
  statistics:
      has_min_max: True
      min: 08f268c60bda420603602149837a7836
      max: 08f489c0368640d9030ab88383a35876
      null_count: 0
      distinct_count: None
      num_values: 10
      physical_type: BYTE_ARRAY
      logical_type: String
      converted_type (legacy): UTF8
  compression: SNAPPY
  encodings: ('PLAIN', 'RLE', 'RLE_DICTIONARY')
  has_dictionary_page: True
  dictionary_page_offset: 4
  data_page_offset: 337
  total_compressed_size: 440
  total_uncompressed_size: 481

In [20]:
type(metadata)

pyarrow._parquet.FileMetaData

In [32]:
# Create a ParquetFile object from the stream
pf = pq.ParquetFile(stream,metadata=metadata)

In [33]:
pf.read_row_group(0)

OSError: Invalid column metadata (corrupt file?)

In [24]:
column = pf.

AttributeError: 'ParquetFile' object has no attribute 'read_column'

In [ ]:
# Read the column
column = pf.read_column(col_index)
columns.append(column)

In [9]:
# Seek to the start of the column chunk
file.seek(column_metadata.file_offset)

# Read the column chunk
column_data = file.read(column_metadata.total_compressed_size)

In [10]:
column_metadata.compression

'SNAPPY'

In [11]:
compression = column_metadata.compression

In [12]:
def decompress_data(data, compression):
    if compression == 'SNAPPY':
        return snappy.decompress(data)
    elif compression == 'GZIP':
        return zlib.decompress(data, wbits=zlib.MAX_WBITS | 16)
    elif compression == 'ZSTD':
        import zstandard
        dctx = zstandard.ZstdDecompressor()
        return dctx.decompress(data)
    elif compression == 'LZ4':
        import lz4.frame
        return lz4.frame.decompress(data)
    elif compression == 'UNCOMPRESSED' or compression is None:
        return data
    else:
        raise ValueError(f"Unsupported compression type: {compression}")

In [13]:
if compression != 'UNCOMPRESSED':
    column_data = decompress_data(column_data, compression)

UncompressError: 

In [16]:
reader = pa.BufferReader(column_data)
column_type = schema.field(column_meta.path_in_schema[0]).type

AttributeError: 'pyarrow._parquet.ParquetSchema' object has no attribute 'field'

In [ ]:
schema.

In [101]:
stream

<pyarrow.BufferReader closed=False own_file=False is_seekable=True is_writable=False is_readable=True>

In [102]:
pf = pq.ParquetFile(stream)

ArrowInvalid: Parquet magic bytes not found in footer. Either the file is corrupted or this is not a parquet file.

In [ ]:
column = pf.read_column(col_index)
columns.append(column)

ArrowInvalid: Parquet magic bytes not found in footer. Either the file is corrupted or this is not a parquet file.

In [85]:
for i in range(row_group.num_columns):
    column = row_group.column(i)
    end_pos = max(end_pos, column.file_offset + column.total_compressed_size)

In [86]:
end_pos

8142

In [87]:
with open(file_path, 'rb') as file:
    file.seek(start_pos)
    row_group_data = file.read(end_pos - start_pos)

In [92]:
reader = pa.BufferReader(row_group_data)

In [93]:
reader

<pyarrow.BufferReader closed=False own_file=False is_seekable=True is_writable=False is_readable=True>

In [ ]:
table = pq.read_table(reader)

ArrowInvalid: Could not open Parquet input source '<Buffer>': Parquet magic bytes not found in footer. Either the file is corrupted or this is not a parquet file.

In [66]:
m.num_rows

10

In [73]:
m.row_group(0).column(0)

  file_offset: 444
  file_path: 
  physical_type: BYTE_ARRAY
  num_values: 10
  path_in_schema: id
  is_stats_set: True
  statistics:
      has_min_max: True
      min: 08f268c60bda420603602149837a7836
      max: 08f489c0368640d9030ab88383a35876
      null_count: 0
      distinct_count: None
      num_values: 10
      physical_type: BYTE_ARRAY
      logical_type: String
      converted_type (legacy): UTF8
  compression: SNAPPY
  encodings: ('PLAIN', 'RLE', 'RLE_DICTIONARY')
  has_dictionary_page: True
  dictionary_page_offset: 4
  data_page_offset: 337
  total_compressed_size: 440
  total_uncompressed_size: 481

In [ ]:
metadata = pf.metadata.row_group(row_group).column(column)

In [48]:
pq.ParquetFile('sample.parquet').read_row_group(0).to_pandas().shape

(10, 15)

In [34]:
md.row_groups

[RowGroup(columns=[ColumnChunk(file_path=None, file_offset=444, meta_data=ColumnMetaData(type=6, encodings=[0, 3, 8], path_in_schema=['id'], codec=1, num_values=10, total_uncompressed_size=481, total_compressed_size=440, key_value_metadata=None, data_page_offset=337, index_page_offset=None, dictionary_page_offset=4, statistics=Statistics(max=None, min=None, null_count=0, distinct_count=None, max_value=b'08f489c0368640d9030ab88383a35876', min_value=b'08f268c60bda420603602149837a7836'), encoding_stats=[PageEncodingStats(page_type=2, encoding=0, count=1), PageEncodingStats(page_type=0, encoding=8, count=1)], bloom_filter_offset=None), offset_index_offset=None, offset_index_length=None, column_index_offset=None, column_index_length=None, crypto_metadata=None, encrypted_column_metadata=None), ColumnChunk(file_path=None, file_offset=857, meta_data=ColumnMetaData(type=6, encodings=[0, 3, 8], path_in_schema=['geometry'], codec=1, num_values=10, total_uncompressed_size=349, total_compressed_siz

In [2]:
import struct
#import snappy
import zlib
from io import BytesIO

In [3]:
file_path = 'sample.parquet'

In [ ]:
f = open(file_path, 'rb')
f.seek(-8, 2)
fl = f.read(4)
struct.unpack('<I', fl)
footer_length = struct.unpack('<I', f.read(4))[0]
f.seek(-footer_length - 8, 2)
footer = f.read(footer_length)
footer

In [ ]:
def parse_metadata(footer):
    # Parse the Thrift-encoded metadata
    # This is a simplified version and may need to be expanded
    metadata = {}
    metadata['schema'] = {'columns': []}
    
    # Extract column information
    # This is a placeholder and needs to be implemented based on the Thrift structure
    
    return metadata

In [ ]:
with open(file_path, 'rb') as f:
    # Read the file footer
    f.seek(-8, 2)
    footer_length = struct.unpack('<I', f.read(4))[0]
    f.seek(-footer_length - 8, 2)
    footer = f.read(footer_length)

    # Parse the footer
    metadata = parse_metadata(footer)

    # Read and decode column data
    data = {}
    for column in metadata['schema']['columns']:
        column_data = read_column(f, metadata, column)
        data[column['name']] = column_data